In [1]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [2]:
import deep_learning

2024-05-22 15:53:40.196371: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 15:53:43.867318: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 15:53:49.642200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
with open("full_data.pic", "rb") as r:
    seeds_total, traces_total, key, output_total, keyshares_total, perms_total = pic.load(r)

In [4]:
X_total, y_total = deep_learning.prepare_data_dl(seeds_total, traces_total, key, keyshares_total, perms_total)

X_profiling, X_extraction = train_test_split(X_total, test_size=100_000, random_state=0)
y_profiling = {}
y_extraction = {}
for label in y_total.keys():
    train, test = train_test_split(y_total[label], test_size=100_000, random_state=0)
    y_profiling[label] = train
    y_extraction[label] = test

In [5]:
try:
    deep_learning.check_file_exists("./resnet_models/resnet_300000.keras")
    from tensorflow.keras.models import load_model
    resnet = load_model("./resnet_models/resnet_300000.keras")
except ValueError:
    resnet = deep_learning.ResNetSCA((X_profiling.shape[1], 1)).model
    

2024-05-22 15:57:51.092325: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-22 15:57:51.092570: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-22 15:57:51.318203: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-22 15:57:51.318273: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-22 15:57:51.318317: I external/local_xla/xla/stream_executor

In [6]:
history = deep_learning.train_model(X_profiling, y_profiling, resnet, "./resnet_models/resnet_300000.keras", epochs=250, patience=25, validation_split=0.1)

Epoch 1/250


I0000 00:00:1716386337.582196  521860 service.cc:145] XLA service 0x7f8ab8011d00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716386337.582281  521860 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-05-22 15:58:58.632194: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-22 15:59:01.089205: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1716386360.562896  521860 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_19', 28 bytes spill stores, 28 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_18', 32 bytes spill stores, 32 bytes spill loads
ptxas warning : Registers are spilled to local memory 

4150/4151 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - block_perm_0_output_accuracy: 0.8834 - loss: 39.6913 - mask_0_0_output_accuracy: 0.0095 - mask_0_1_output_accuracy: 0.0096 - mask_0_2_output_accuracy: 0.0098 - mask_0_3_output_accuracy: 0.0095 - mask_0_4_output_accuracy: 0.0092 - mask_0_5_output_accuracy: 0.0092 - mask_0_6_output_accuracy: 0.0099 - round_perm_output_accuracy: 0.5273

2024-05-22 16:08:22.128738: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 32.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-22 16:08:22.221993: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


4151/4151 ━━━━━━━━━━━━━━━━━━━━ 601s 137ms/step - block_perm_0_output_accuracy: 0.8834 - loss: 39.6913 - mask_0_0_output_accuracy: 0.0095 - mask_0_1_output_accuracy: 0.0096 - mask_0_2_output_accuracy: 0.0098 - mask_0_3_output_accuracy: 0.0095 - mask_0_4_output_accuracy: 0.0092 - mask_0_5_output_accuracy: 0.0092 - mask_0_6_output_accuracy: 0.0099 - round_perm_output_accuracy: 0.5273 - val_block_perm_0_output_accuracy: 0.6894 - val_loss: 42.5252 - val_mask_0_0_output_accuracy: 0.0055 - val_mask_0_1_output_accuracy: 0.0043 - val_mask_0_2_output_accuracy: 0.0073 - val_mask_0_3_output_accuracy: 0.0069 - val_mask_0_4_output_accuracy: 0.0069 - val_mask_0_5_output_accuracy: 0.0067 - val_mask_0_6_output_accuracy: 0.0067 - val_round_perm_output_accuracy: 0.3793
Epoch 2/250
4151/4151 ━━━━━━━━━━━━━━━━━━━━ 528s 127ms/step - block_perm_0_output_accuracy: 0.8866 - loss: 39.6609 - mask_0_0_output_accuracy: 0.0098 - mask_0_1_output_accuracy: 0.0094 - mask_0_2_output_accuracy: 0.0094 - mask_0_3_output_ac

In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

y_predicted = load_model("./resnet_models/resnet_350000_52_epochs.keras").predict(X_extraction)
for i, k in enumerate(y_extraction.keys()):
    print(f"{k}: {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k])}/{y_extraction[k].shape[0]} = {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k]) / y_extraction[k].shape[0]}")

np.mean([categorical_crossentropy(to_categorical(y_extraction[k]), y_predicted[i]) for i, k in enumerate(y_extraction.keys())], axis=1)

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step
x_round: 24419/45133 = 0.5410453548401392
block_perm_0: 39134/45133 = 0.8670817362018922
mask_0_0: 354/45133 = 0.007843484811556953
mask_0_1: 365/45133 = 0.00808720891587087
mask_0_2: 305/45133 = 0.006757804710522234
mask_0_3: 302/45133 = 0.006691334500254802
mask_0_4: 332/45133 = 0.0073560366029291205
mask_0_5: 305/45133 = 0.006757804710522234
mask_0_6: 276/45133 = 0.006115259344603727


array([1.3536543, 0.3749589, 5.503599 , 5.5072737, 5.477    , 5.5458245,
       5.490437 , 5.568437 , 5.656007 ], dtype=float32)